In [2]:
import numpy as np
import pandas as pd
from pylab import plt
import scipy.optimize as sco
plt.style.use("ggplot")

In [46]:
def find_covariance_matrix(corr, st_dev):
    n = len(st_dev)
    cov_matrix = np.zeros((n, n))
    
    # Indexing the correlations based on the provided order [ρ12, ρ23, ρ13]
    corr_index = { (0, 1): 0, (1, 2): 1, (0, 2): 2 }
    
    # Fill the upper triangle and the lower triangle symmetrically
    for i in range(n):
        for j in range(i + 1, n):
            rho = corr[corr_index[(i, j)] if (i, j) in corr_index else corr_index[(j, i)]]
            cov_matrix[i, j] = rho * st_dev[i] * st_dev[j]
            cov_matrix[j, i] = cov_matrix[i, j]

    # Fill the diagonal
    for i in range(n):
        cov_matrix[i, i] = st_dev[i] ** 2

    return cov_matrix

def invert_matrix(matrix):
    return np.linalg.inv(matrix)

def find_mvp_weights(inverted_cov_mat, st_dev):
    u = np.ones(len(st_dev))
    u_C_inv = u @ inverted_cov_mat
    u_C_inv_uT = u_C_inv @ u.T
    
    mvp_weights = []
    for i in u_C_inv:
        mvp_weights.append(i / u_C_inv_uT)
    
    return mvp_weights

In [40]:
expected_returns = [0.15, -0.1, 0.1]
standard_deviations = [0.2, 0.15, 0.1]
correlations = [0.3, -0.1, 0.2] # [ρ12, ρ23, ρ13]

### Find Covariance Matrix (C) and Inverse (C-1)

In [41]:
cov_mat = find_covariance_matrix(correlations, standard_deviations)
cov_mat_inv = invert_matrix(cov_mat)

print(cov_mat)
print("\n")
print(cov_mat_inv)

[[ 0.04    0.009   0.004 ]
 [ 0.009   0.0225 -0.0015]
 [ 0.004  -0.0015  0.01  ]]


[[ 29.18632075 -12.57861635 -13.56132075]
 [-12.57861635  50.31446541  12.57861635]
 [-13.56132075  12.57861635 107.31132075]]


### Find Weights of MVP

In [42]:
mvp_weights = find_mvp_weights(cov_mat_inv, standard_deviations)
print(mvp_weights)

[0.01907692307692311, 0.31507692307692303, 0.6658461538461538]


### Compute MVP Return

In [43]:
expected_returns_array = np.array(expected_returns)

mvp_return = mvp_weights @ expected_returns_array.T
print(mvp_return)

0.03793846153846154


### Compute MVP Variance

In [44]:
mvp_weights_array = np.array(mvp_weights)

mvp_variance = mvp_weights_array @ cov_mat @ mvp_weights_array.T
print(mvp_variance)

0.006262153846153846


### Find MVP with Return Constraint (10%)

In [51]:
u = np.ones(len(standard_deviations))

m_C_inv = expected_returns_array @ cov_mat_inv
m_C_inv_mT = m_C_inv @ expected_returns_array.T
m_c_inv_uT = m_C_inv @ u.T
u_C_inv_uT = u @ cov_mat_inv @ u.T
u_C_inv_mT = u @ cov_mat_inv @ expected_returns_array.T

print(m_C_inv)
print(m_C_inv_mT)
print(m_c_inv_uT)
print(u_C_inv_uT)
print(u_C_inv_mT)

[ 4.27967767 -5.66037736  7.43907233]
1.951896619496855
6.058372641509432
159.689465408805
6.0583726415094326


### Find Lambdas

In [52]:
return_constraint = 0.1

lambda_2 = ((2 * return_constraint * m_c_inv_uT) - 2 * m_C_inv_mT) / ((m_c_inv_uT ** 2) - (u_C_inv_uT * m_C_inv_mT))
lambda_1 = ((2 * return_constraint) - (lambda_2 * u_C_inv_mT)) / m_C_inv_mT

print(lambda_2)
print(lambda_1)

0.009789755807027994
0.07207861822513403


### Find Weights

In [53]:
ret_cons_weights = ((lambda_1 / 2) * (expected_returns_array @ cov_mat_inv)) + ((lambda_2 / 2) * (u @ cov_mat_inv))

print(ret_cons_weights)

[0.1691483  0.04228708 0.78856462]


### Find Portfolio Variance

In [56]:
ret_cons_variance = ret_cons_weights @ cov_mat @ ret_cons_weights.T
print(ret_cons_variance)

0.008498808814770695


### Confirm 10% Return

In [65]:
ret_cons_return = ret_cons_weights @ expected_returns_array.T
print(ret_cons_return)

0.09999999999999999


### Find Market Portfolio for 0 < R < μMVP

In [64]:
R = 0.02 # μMVP = 0.38, R is risk-free rate.

mp_weights = ((expected_returns_array - (R * u)) @ cov_mat_inv) / ((expected_returns_array - (R * u)) @ cov_mat_inv @ u.T)
print(mp_weights)

# Negative weight denotes short position.

[ 1.47272727 -2.32727273  1.85454545]


### Market Portfolio Return

In [66]:
mp_return = mp_weights @ expected_returns_array.T
print(mp_return)

0.6390909090909095


### Beta, Expected Return & Risk Premium for given Portfolio

In [71]:
given_weights = np.array([0.2, 0.3, 0.5])

# Beta = Cov(km, kw) / Market Portfolio Variance

mp_variance = mp_weights @ cov_mat @ mp_weights.T
print(mp_variance)

0.2161190082644631


In [72]:
# Cov(km, kw) = W * C * WmT

cov_km_kw = given_weights @ cov_mat @ mp_weights.T
print(cov_km_kw)

0.010472727272727278


### Beta:

In [73]:
given_beta = cov_km_kw / mp_variance
print(given_beta)

0.04845814977973564


### Expected Return:

In [77]:
# Expected Return = R + Beta * (Market Portfolio Return + R)

given_exp_return = (R + given_beta) * (mp_return + R)
print(given_exp_return)

0.045120144173007616


### Risk Premium:

In [78]:
given_risk_premium = given_beta * (mp_return - R)
print(given_risk_premium)

0.029999999999999995
